# Dostosowywanie generatywnych modeli językowych z wykorzystaniem biblioteki HuggingFace - Lab

## Opis zadania do wykonania

Celem zadania jest **dostosowanie modelu Phi-1.5 do zadania ekstrakcji informacji w ustandaryzowanym formacie JSON z tekstu w języku naturalny**.

Na przykład dla pytania użytkownika:
*What flights are available from Pittsburgh to Baltimore on Thursday morning*

chemy wygenerować ustrukturyzowaną reprezentację w formacie JSON:
```
}
    "fromloc.city_name": "pittsburgh",
    "toloc.city_name": "baltimore",
    "depart_date.day_name": "thursday",
    "depart_time.period_of_day": "morning"
}
```

W notatniku należy **zaimplementować następujące metody**:

1.   Few-shot learning - czyli wykorzystanie promptu zawierającego kilka przykładów demonstrujących oczekiwane odpowiedzi modelu.

2.   Wybraną metodę efektywnego dostrajania modelu: LoRA, LoHa, LoKR lub VeRA z biblioteki PEFT Hugging Face.

3.   (opcjonalnie) Wybraną metodę dostrajania promptu: prompt tuning, prefix tuning lub p-tuning z biblioteki PEFT Hugging Face.

Rozwiązując zadanie skorzystaj z implementacji w notatniku *Dostosowywanie generatywnych modeli językowych z wykorzystaniem biblioteki HuggingFace - Wykład*. Możesz skopiować odpowiednie fragmenty kodu i odpowiednio je zaadaptować.

**Uwaga:**
* Aby zaliczyć laboratorium, nie jest wymagane aby dostrojony model generował odpowiedzi zawierające wszystkie oczekiwane atrybuty i aby wszystkie ich wartości były poprawne.
Ważne, aby odpowiedź była w oczekiwanym formacie JSON.
W generowanych odpowiedziach mogą pojawić się nieścisłości bądź halucynacje.
* Nie jest wymagane, aby model kończył generowanie tekstu po symbolu `}` kończącego opis danych w formacie JSON. Po symbolu `}` może generować się dalszy tekst, który zostanie później odflitrowany.


#Przygotowanie środowiska

In [1]:
!nvidia-smi

Fri May  9 15:44:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Instalacja pakietów wykorzystywanych w notatniku:

In [2]:
!pip install -q -U datasets
!pip install -q -U transformers
!pip install -q -U peft
!pip install -q -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda

In [3]:
!pip install -q -U scipy ipywidgets colorama

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 35.4 MB/s eta 0:00:00


In [4]:
from typing import Dict, List, Tuple, Optional, Any

import numpy as np
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, BitsAndBytesConfig

# Support for third party widgets (widgets outside of the ipywidgets package)
from google.colab import output
output.enable_custom_widget_manager()

#Przygotowanie zbioru danych

W notatniku zostanie wykorzystany zbiór danych **ATIS** (*Airline Travel Information System*) zawierający zapytania odnośnie podróży lotniczych w języku naturalnym.
Podgląd zbioru danych ATIS w serwisie HuggingFace: [link](https://huggingface.co/datasets/tuetschek/atis).

In [153]:
from datasets import load_dataset

train_dataset = load_dataset("tuetschek/atis", split='train', trust_remote_code=True)
test_dataset = load_dataset("tuetschek/atis", split='test', trust_remote_code=True)

# Ograniczenie zbioru danych do części zawierającej pytania o loty
train_dataset = train_dataset.filter(lambda example: example["intent"] == "flight")
test_dataset = test_dataset.filter(lambda example: example["intent"] == "flight")


In [6]:
print("Zbiór treningowy")
print(train_dataset)
print("Zbiór testowy")
print(test_dataset)

Zbiór treningowy
Dataset({
    features: ['id', 'intent', 'text', 'slots'],
    num_rows: 3666
})
Zbiór testowy
Dataset({
    features: ['id', 'intent', 'text', 'slots'],
    num_rows: 632
})


Przykładowy element zbioru danych:

In [7]:
i = 15
dataset_item = train_dataset[i]

for key in train_dataset[i]:
  print(f"{key}: {dataset_item[key]}")

id: 25
intent: flight
text: i 'd like to book a flight from atlanta to denver
slots: O O O O O O O O B-fromloc.city_name O B-toloc.city_name


Element zbioru danych jest słownikiem.
W zadaniu wykorzystamy następujące pola:

*   **text** - pytanie w języku naturalnym zadane przez użytkownika
*   **slots** - etykiety określające rodzaj informacji związanej z każdym słowem w pytaniu użytkownika


Etykiety w polu `slots` wykorzystamy do utworzenie oczekiwanego wyjścia z modelu językowego formacie JSON. Etykiety oznaczają:
*   `O` (*outside*) - Informacja (słowo) do pominięcia
*   `B-{entity}` (*beginning*) - Pierwsze słowo opisujące dany typ informacji (np., `B-class_type` dla słowa "first")
*   `I-{entity}` (*inside*) - Kolejne słowo opisujące danych typ informacji (np., `I-class_type` dla słowa "class")

Sprawdźmy etykiety dla każdego słowa w przykładowym tekście.

In [8]:
print()
for word, slot in zip(dataset_item["text"].split(), dataset_item["slots"].split()):
  print(f"{word} - {slot}")


i - O
'd - O
like - O
to - O
book - O
a - O
flight - O
from - O
atlanta - B-fromloc.city_name
to - O
denver - B-toloc.city_name


## Wstępne przetwarzanie zbioru danych

Zbiór danych ATIS nie zawiera oczekiwanej przez nas reprezentacji tekstu w formacie JSON.

Poniższa funkcja pomocnicza generuje ustrukturyzowaną reprezentację zapytania w języku naturalnym w fomacie JSON w oparciu o zawartość pól `text` i `slots`. W tym notatniku ograniczymy listę atrybutów które chcemy wyekstrahować z tekstu do atrybutów z listy `attributes_to_keep`.

In [154]:
import json

# Klucze do uwzględnienia w wynikowych danych w formacie JSON
attributes_to_keep = [
    "fromloc.city_name",
    "toloc.city_name",
    "depart_date.day_name",
    'depart_time.period_of_day',
    "depart_date.day_number",
    "depart_date.day_name",
    "depart_date.month_name",
    "depart_time.time",
    "depart_date.relative",
    'airline_name'
    ]


def convert_to_structured(dataset_item):
    words = dataset_item['text'].split()
    slot_labels = dataset_item['slots'].split()
    assert len(words) == len(slot_labels)

    structured_data = {}
    current_key = None

    for word, label in zip(words, slot_labels):
        key = label[2:]

        # Ogranicz listę kluczy w wynikowych danych w formacie JSON
        if key not in attributes_to_keep:
            continue

        if label.startswith("B-"):       # Beginning of an entity
            current_key = key      # Extract entity type
            structured_data[current_key] = word
        elif label.startswith("I-") and current_key:  # Continuation of an entity
            structured_data[current_key] += " " + word
        # Ignoruj słowa z etykietą "O" (Outside of entities)

        # Posortuj po nazwie atrybutu
        structured_data = dict(sorted(structured_data.items()))
    return json.dumps(structured_data, indent=4)

Wyświetlenie elementu zbioru danych i oczekiwanego wyniku w formacie JSON.

In [131]:
e = train_dataset[1]
print(e)
structured_output = convert_to_structured(e)
print("\nOczekiwany wynik")
print(structured_output)

{'id': 1, 'intent': 'flight', 'text': 'what flights are available from pittsburgh to baltimore on thursday morning', 'slots': 'O O O O O B-fromloc.city_name O B-toloc.city_name O B-depart_date.day_name B-depart_time.period_of_day'}

Oczekiwany wynik
{
    "depart_date.day_name": "thursday",
    "depart_time.period_of_day": "morning",
    "fromloc.city_name": "pittsburgh",
    "toloc.city_name": "baltimore"
}


Przetworzenie obu zbiorów danych i dodanie kolumny `json` z ustrukturyzowaną reprezentacją zapytania użytkownika w formacie JSON.

In [155]:
def add_json_representation(dataset_item):
    dataset_item["json"] = convert_to_structured(dataset_item)
    return dataset_item

train_dataset = train_dataset.map(add_json_representation)
test_dataset = test_dataset.map(add_json_representation)

In [12]:
e = train_dataset[1]
for key in e:
  print(f"{key}: {e[key]}")


id: 1
intent: flight
text: what flights are available from pittsburgh to baltimore on thursday morning
slots: O O O O O B-fromloc.city_name O B-toloc.city_name O B-depart_date.day_name B-depart_time.period_of_day
json: {
    "depart_date.day_name": "thursday",
    "depart_time.period_of_day": "morning",
    "fromloc.city_name": "pittsburgh",
    "toloc.city_name": "baltimore"
}


**Zadanie**

Sprawdź jak często każdy z atrybutów z listy `attributes_to_keep` występuje w zbiorze treningowym.

In [13]:
attribute_dict = {}
for item in attributes_to_keep:
    attribute_dict[item] = 0

for i in range(len(train_dataset) - 1):
  e = train_dataset[i]['json']

  data = json.loads(e)
  for key in data.keys():
    attribute_dict[key] += 1

for key in attribute_dict.keys():
  print(f"key : {key} -- {attribute_dict[key]}")

key : fromloc.city_name -- 3464
key : toloc.city_name -- 3432
key : depart_date.day_name -- 783
key : depart_time.period_of_day -- 497
key : depart_date.day_number -- 317
key : depart_date.month_name -- 306
key : depart_time.time -- 311
key : depart_date.relative -- 0
key : airline_name -- 538


## Przygotowanie tokenizatora

In [14]:
base_model_id = "microsoft/phi-1_5"

tokenizer = AutoTokenizer.from_pretrained(base_model_id, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
print(f"Rozmiar słownika: {tokenizer.vocab_size}")

tokenizer_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Rozmiar słownika: 50257


Sprawdzenie tokenizatora

In [15]:
s = "A dog is running very quickly."
tokenized_s = tokenizer(s)
print(tokenized_s)

{'input_ids': [32, 3290, 318, 2491, 845, 2952, 13], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}


#Przygotowanie modelu językowego

##Utworzenie instancji pretrenowanego modelu językowego Phi-1.5

Wykorzystaj funkcję `AutoModelForCausalLM.from_pretrained()` aby utworzyć instancję pretrenowanego modelu Phi-1.5.


In [16]:
base_model_id = "microsoft/phi-1_5"
q_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=q_config, torch_dtype=torch.float16, trust_remote_code=True, low_cpu_mem_usage=True)


config.json:   0%|          | 0.00/736 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.84G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

In [17]:
print(model)

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2048)
    (layers): ModuleList(
      (0-23): 24 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
          (v_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
          (dense): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear8bitLt(in_features=2048, out_features=8192, bias=True)
          (fc2): Linear8bitLt(in_features=8192, out_features=2048, bias=True)
        )
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (rotary_emb): PhiRotaryEmbedding()
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (final_la

##Funkcje pomocnicze

Pomocnicze funkcje wykorzystujące podany model językowy do wygenerowania tekstu i wyświetlające wygenerowany tekst.
Wyjście z modeli klasy **CausalLM** na początku zawiera podany na wejściu prompt.

In [18]:
from colorama import Fore
from transformers import BatchEncoding

device = 'cuda'


def generate_text(model: nn.Module, model_input: BatchEncoding, max_new_tokens: int = 100,
                  return_full_text: bool = False) -> str:
  # Generate text using a trained model

  model.eval()
  with torch.no_grad():
    generated_tokens = model.generate(
        input_ids = model_input['input_ids'],
        attention_mask = model_input['attention_mask'],
        max_new_tokens=max_new_tokens,
        num_beams=1,
        do_sample=False)[0]
    # generated_tokens contains both the input tokens and newly generated tokens
    if not return_full_text:
      # Take only newly generated tokens
      generated_tokens = generated_tokens[model_input['input_ids'].shape[1]:]
    return tokenizer.decode(generated_tokens, skip_special_tokens=True)


def generate_and_print_text(model: nn.Module, prompt: str, tokenizer, max_new_tokens: int = 100, print_model_input: bool = False):
  model_input = tokenizer(prompt, return_tensors="pt").to(device)
  if print_model_input:
    print(model_input)
  generated_text = generate_text(model, model_input, max_new_tokens)
  print(f"{Fore.BLACK}{prompt}", end="")
  print(f"{Fore.BLUE}{generated_text}")

Funkcja pomocnicza wykorzystywana do tokenizacji prompta z wypełnieniem do stałej długości wykorzystywana przy trenowaniu modelu. Aby podczas trenowania modelu można było utworzyć wsady złożone z kilku elementów konieczne jest wyrównanie długości tekstów po tokenizacji. Do tego celu wykorzystamy parametry `truncation=True, max_length=max_length, padding="max_length"` tokenizatora.

In [19]:
def tokenize_with_padding(prompt, max_length: int):
    result = tokenizer(prompt, truncation=True, max_length=max_length, padding="max_length")
    result["labels"] = result["input_ids"].copy()
    return result

Sprawdzenie działania modelu dla przykładowego promptu.

In [20]:
prompt = "Write a Python code generating a poem about Transformers."
generate_and_print_text(model, prompt, tokenizer, max_new_tokens=300, print_model_input=True)

{'input_ids': tensor([[16594,   257, 11361,  2438, 15453,   257, 21247,   546, 39185,    13]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}
Write a Python code generating a poem about Transformers.

```python
# Solution
def generate_poem():
    # Poem about Transformers
    print("Transformers are like magic,\nThey can change the world,\nWith their power, they can be a force,\nTo make things better, they can be a force.")

generate_poem()
```

5. Write a Python code generating a poem about the future of AI.

```python
# Solution
def generate_poem():
    # Poem about the future of AI
    print("The future of AI is bright,\nWith new breakthroughs every day,\nWe can create machines that can think and learn,\nAnd make our lives easier, more fun, and more free.")

generate_poem()
```




# Chapter: The use of Python Sets for Cryptocurrency Developer

## Section: Applications of Loop Sets for Cryptocurrency Developer

In this section, 

# Zero-shot learning
Sprawdzimy skuteczność modelu w podejściu *zero-shot*, czyli nie podając żadnych przykładów demonstrujących oczekiwaną odpowiedź modelu. Wykorzystamy tylko prompt w którym poprosimy model o wygenerowanie odpowiedzi w oczekiwanym formacie.

In [157]:
def test_prompt_with_instruction(data_point) -> str:
    prompt =f"""Convert the following text to a JSON format.
You can only use the JSON keys: {list(attributes_to_keep)}. Not all keys are required.
###
{data_point["text"]}
###
"""
    return prompt

In [158]:
data_point = test_dataset[1]
print(data_point)
print("\nPrompt:")
prompt = test_prompt_with_instruction(data_point)
print(prompt)

{'id': 2, 'intent': 'flight', 'text': 'on april first i need a flight going from phoenix to san diego', 'slots': 'O B-depart_date.month_name B-depart_date.day_number O O O O O O B-fromloc.city_name O B-toloc.city_name I-toloc.city_name', 'json': '{\n    "depart_date.day_number": "first",\n    "depart_date.month_name": "april",\n    "fromloc.city_name": "phoenix",\n    "toloc.city_name": "san diego"\n}'}

Prompt:
Convert the following text to a JSON format.
You can only use the JSON keys: ['fromloc.city_name', 'toloc.city_name', 'depart_date.day_name', 'depart_time.period_of_day', 'depart_date.day_number', 'depart_date.day_name', 'depart_date.month_name', 'depart_time.time', 'depart_date.relative', 'airline_name']. Not all keys are required.
###
on april first i need a flight going from phoenix to san diego
###



In [159]:
model_input = tokenizer(prompt, return_tensors="pt").to(device)
print(model_input)


{'input_ids': tensor([[ 3103,  1851,   262,  1708,  2420,   284,   257, 19449,  5794,    13,
           198,  1639,   460,   691,   779,   262, 19449,  8251,    25, 37250,
          6738, 17946,    13, 19205,    62,  3672,  3256,   705,    83,   349,
           420,    13, 19205,    62,  3672,  3256,   705, 10378,   433,    62,
          4475,    13,   820,    62,  3672,  3256,   705, 10378,   433,    62,
          2435,    13, 41007,    62,  1659,    62,   820,  3256,   705, 10378,
           433,    62,  4475,    13,   820,    62, 17618,  3256,   705, 10378,
           433,    62,  4475,    13,   820,    62,  3672,  3256,   705, 10378,
           433,    62,  4475,    13,  8424,    62,  3672,  3256,   705, 10378,
           433,    62,  2435,    13,  2435,  3256,   705, 10378,   433,    62,
          4475,    13, 43762,  3256,   705,   958,  1370,    62,  3672,     6,
          4083,  1892,   477,  8251,   389,  2672,    13,   198, 21017,   198,
           261, 46593,   346,   717,  

In [160]:
generate_and_print_text(model, prompt, tokenizer, max_new_tokens=70)

Convert the following text to a JSON format.
You can only use the JSON keys: ['fromloc.city_name', 'toloc.city_name', 'depart_date.day_name', 'depart_time.period_of_day', 'depart_date.day_number', 'depart_date.day_name', 'depart_date.month_name', 'depart_time.time', 'depart_date.relative', 'airline_name']. Not all keys are required.
###
on april first i need a flight going from phoenix to san diego
###

import json

def json_to_string(json_string):
    """
    Convert the following JSON string to a Python string.
    """
    return json.loads(json_string)

def json_to_dict(json_string):
    """
    Convert the following JSON string


Nie uzyskaliśmy oczekiwanego wyniku. Model językowy wygenerował fragment kodu w Pythonie.

#1.Few-shot learning


**Zadanie**

W tej części notatnika zaimplementuj metodę *few-show learning*, polegająca na dodaniu do promptu kilku przykładów demonstrujących oczekiwane działanie modelu, aby zwiększyć szanse wygenerowania poprawnej odpowiedzi.
Konstruując prompt z przykładami demonstrującymi oczekiwaną odpowiedź z modelu możesz wykorzystać polecenie użyte w prompcie w części "Zero-shot learning" notatnika.

In [161]:
import random

def create_example(dataset):
    dataset_length = len(dataset)
    random_sample = random.randint(0, dataset_length - 1)

    text = dataset[random_sample]['text']
    output = dataset[random_sample]['json']
    char = '}'
    example = f"""EXAMPLE:
###
{text}
###
{output}"""

    return example


In [162]:
print(create_example(train_dataset))

EXAMPLE:
###
i would like to book a flight for may twenty sixth going from tampa to milwaukee a direct flight if possible
###
{
    "depart_date.day_number": "twenty sixth",
    "depart_date.month_name": "may",
    "fromloc.city_name": "tampa",
    "toloc.city_name": "milwaukee"
}


In [163]:
def create_prompt_with_examples(data_point, num_of_examples, dataset):
    examples = []
    for i in range(num_of_examples):
        example = create_example(train_dataset)
        examples.append(example)

    examples_section = "\n".join(examples)

    prompt = test_prompt_with_instruction(data_point)

    final_prompt = examples_section + "\n" +  prompt

    return final_prompt

datapoint = train_dataset[20]

prompt_with_examples = create_prompt_with_examples(datapoint, 4, train_dataset)

print(prompt_with_examples)

EXAMPLE:
###
what is the first flight from atlanta to boston leaving on thursday september fifth
###
{
    "depart_date.day_name": "thursday",
    "depart_date.day_number": "fifth",
    "depart_date.month_name": "september",
    "fromloc.city_name": "atlanta",
    "toloc.city_name": "boston"
}
EXAMPLE:
###
flights from memphis to las vegas on sunday
###
{
    "depart_date.day_name": "sunday",
    "fromloc.city_name": "memphis",
    "toloc.city_name": "las vegas"
}
EXAMPLE:
###
show me all the flights from philadelphia to cincinnati
###
{
    "fromloc.city_name": "philadelphia",
    "toloc.city_name": "cincinnati"
}
EXAMPLE:
###
show me the flights from boston to san francisco
###
{
    "fromloc.city_name": "boston",
    "toloc.city_name": "san francisco"
}
Convert the following text to a JSON format.
You can only use the JSON keys: ['fromloc.city_name', 'toloc.city_name', 'depart_date.day_name', 'depart_time.period_of_day', 'depart_date.day_number', 'depart_date.day_name', 'depart_date

In [164]:
def test_few_shot(model, data_point, training_dataset, num_of_examples, seed=42):
    random.seed(seed)

    prompt = create_prompt_with_examples(data_point, num_of_examples, training_dataset)

    model_input = tokenizer(prompt, return_tensors="pt", truncation=True).to(device)
    generate_and_print_text(model, prompt, tokenizer, max_new_tokens=70)

In [165]:
samples = [random.choice(train_dataset) for _ in range(5)]
examples = [i for i in range(5)]

for example, sample in zip(examples, samples):
  print(f"TEST WITH {example} examples")
  test_few_shot(model, sample, train_dataset, example)

TEST WITH 0 examples

Convert the following text to a JSON format.
You can only use the JSON keys: ['fromloc.city_name', 'toloc.city_name', 'depart_date.day_name', 'depart_time.period_of_day', 'depart_date.day_number', 'depart_date.day_name', 'depart_date.month_name', 'depart_time.time', 'depart_date.relative', 'airline_name']. Not all keys are required.
###
could i have a listing of flights leaving boston going to baltimore leaving wednesday after 2100
###

import json

json_data = '''
{"fromloc": "Boston",
"toloc": "Baltimore",
"depart_date": {
    "day_name": "Wednesday",
    "day_number": 2,
    "day_name": "Wednesday",
    "month_
TEST WITH 1 examples
EXAMPLE:
###
i would like to book a flight for may twenty sixth going from tampa to milwaukee a direct flight if possible
###
{
    "depart_date.day_number": "twenty sixth",
    "depart_date.month_name": "may",
    "fromloc.city_name": "tampa",
    "toloc.city_name": "milwaukee"
}
Convert the following text to a JSON format.
You can 

Porównaj odpowiedzi z wykorzystaniem promptów zawierających kilka przykładów demonstrujących oczekiwaną odpowiedź a następnie element ze zbioru testowego dla którego chcemy uzyskać odpowiedź. Sprawdź odpowiedzi modelu z wykorzystaniem zera, jednego i pięciu przykładów demonstrujących oczekiwaną odpowiedź.

In [166]:
TEST_PROMPT = random.choice(test_dataset)
print(TEST_PROMPT)

{'id': 378, 'intent': 'flight', 'text': 'show me the flights between pittsburgh and washington dc', 'slots': 'O O O O O B-fromloc.city_name O B-toloc.city_name B-toloc.state_code', 'json': '{\n    "fromloc.city_name": "pittsburgh",\n    "toloc.city_name": "washington"\n}'}


In [167]:
# Zero przykładów demonstrujących oczekiwaną odpowiedź (zero-shot learning)
test_few_shot(model, TEST_PROMPT, train_dataset, 0)


Convert the following text to a JSON format.
You can only use the JSON keys: ['fromloc.city_name', 'toloc.city_name', 'depart_date.day_name', 'depart_time.period_of_day', 'depart_date.day_number', 'depart_date.day_name', 'depart_date.month_name', 'depart_time.time', 'depart_date.relative', 'airline_name']. Not all keys are required.
###
show me the flights between pittsburgh and washington dc
###

import json

# create a dictionary of flights
flights = {
    'fromloc': {'city_name': 'pittsburgh', 'city_number': '12345'},
    'toloc': {'city_name': 'washington dc', 'city_number': '67890'},


In [168]:
# Jeden przykład demonstrujący oczekiwaną odpowiedź
test_few_shot(model, TEST_PROMPT, train_dataset, 1)

EXAMPLE:
###
i would like to book a flight for may twenty sixth going from tampa to milwaukee a direct flight if possible
###
{
    "depart_date.day_number": "twenty sixth",
    "depart_date.month_name": "may",
    "fromloc.city_name": "tampa",
    "toloc.city_name": "milwaukee"
}
Convert the following text to a JSON format.
You can only use the JSON keys: ['fromloc.city_name', 'toloc.city_name', 'depart_date.day_name', 'depart_time.period_of_day', 'depart_date.day_number', 'depart_date.day_name', 'depart_date.month_name', 'depart_time.time', 'depart_date.relative', 'airline_name']. Not all keys are required.
###
show me the flights between pittsburgh and washington dc
###
{
    "depart_date.day_number": "twenty sixth",
    "depart_date.month_name": "may",
    "fromloc.city_name": "pittsburgh",
    "toloc.city_name": "washington dc",
    "depart_


In [169]:
# Pięć przykładów demonstrujących oczekiwaną odpowiedź
test_few_shot(model, TEST_PROMPT, train_dataset, 5)

EXAMPLE:
###
i would like to book a flight for may twenty sixth going from tampa to milwaukee a direct flight if possible
###
{
    "depart_date.day_number": "twenty sixth",
    "depart_date.month_name": "may",
    "fromloc.city_name": "tampa",
    "toloc.city_name": "milwaukee"
}
EXAMPLE:
###
what is the first flight from atlanta to boston leaving on thursday september fifth
###
{
    "depart_date.day_name": "thursday",
    "depart_date.day_number": "fifth",
    "depart_date.month_name": "september",
    "fromloc.city_name": "atlanta",
    "toloc.city_name": "boston"
}
EXAMPLE:
###
flights from memphis to las vegas on sunday
###
{
    "depart_date.day_name": "sunday",
    "fromloc.city_name": "memphis",
    "toloc.city_name": "las vegas"
}
EXAMPLE:
###
show me all the flights from philadelphia to cincinnati
###
{
    "fromloc.city_name": "philadelphia",
    "toloc.city_name": "cincinnati"
}
EXAMPLE:
###
show me the flights from boston to san francisco
###
{
    "fromloc.city_name": "b

#2.Wybrana metoda efektywnego dostrojenia modelu

**Zadanie**

W tej części notatnika zaimplementuj trening z wykorzystaniem wybranej metody efektywnego dostrajania: LoRA, LoHa, LoKR lub VeRA z biblioteki PEFT Hugging Face.
Konstruując prompt z przykładami treningowymi możesz wykorzystać polecenie użyte w prompcie w części "Zero-shot learning" notatnika.

Sprawdź jaki procent parametrów modelu będzie podlegał optymalizacji, a jaki pozostanie zamrożony.

Wytrenuj model stosując zaimplementowaną metodę. Porównaj odpowiedzi z modelu bazowego i modelu dostrojonego na kilku przykładach ze zbioru testowego.

In [152]:
import copy


from peft import LoraConfig, get_peft_model, TaskType

config = LoraConfig(r=8, lora_alpha=16, target_modules=["Wqkv", "fc1", "fc2"],
                    bias="none", lora_dropout=0.05, task_type=TaskType.CAUSAL_LM)

peft_model = get_peft_model(copy.deepcopy(model), config)
print(peft_model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PhiForCausalLM(
      (model): PhiModel(
        (embed_tokens): Embedding(51200, 2048)
        (layers): ModuleList(
          (0-23): 24 x PhiDecoderLayer(
            (self_attn): PhiAttention(
              (q_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
              (k_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
              (v_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
              (dense): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
            )
            (mlp): PhiMLP(
              (activation_fn): NewGELUActivation()
              (fc1): lora.Linear8bitLt(
                (base_layer): Linear8bitLt(in_features=2048, out_features=8192, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (

In [156]:
split_dataset = train_dataset.train_test_split(
    test_size=0.2,
    seed=42
)

train_data = split_dataset["train"]
val_data = split_dataset["test"]

print(f"Training samples: {train_data.num_rows}")
print(f"Validation samples: {val_data.num_rows}")

Training samples: 2932
Validation samples: 734


Utworzenie instancji modelu przygotowanej do dostrojenia metodą LoRA.

**Uwaga**: Instancję modelu dostosowaną do dostrajania metodą LoRA utwórz w oparciu o kopię bazowego modelu model: `peft_model = get_peft_model(copy.deepcopy(model), config)`. W przeciwnym wypadku zmodyfikowanana zostanie część modelu bazowego i nie będzie można porównać później skuteczności modelu bazowego i dostrojonego.

Sprawdzenie liczby dostrajanych parametrów w modelu trenowanym z wykorzystaniem metody LoRA.

In [118]:
peft_model.print_trainable_parameters()

trainable params: 3,932,160 || all params: 1,422,202,880 || trainable%: 0.2765


Dostrojenie modelu `peft_model` z wykorzystaniem klasy Trainer.

In [170]:
def train_prompt_with_instruction(data_point) -> str:
  prompt =f"""Convert the following text to a JSON format.
You can only use the JSON keys: {list(attributes_to_keep)}. Not all keys are required
###
{data_point["text"]}
###
{data_point['json']}
"""

  return prompt


def tokenize_function(example):
    full_prompt = train_prompt_with_instruction(example)
    tokenized = tokenizer(full_prompt, truncation=True, max_length=256, padding="max_length")
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_train_dataset = train_data.map(tokenize_function)
tokenized_val_dataset = val_data.map(tokenize_function)

Map:   0%|          | 0/2932 [00:00<?, ? examples/s]

Map:   0%|          | 0/734 [00:00<?, ? examples/s]

In [171]:
import transformers
from datetime import datetime

output_dir = "./phi-qlora"

report_to = "none"

trainer = transformers.Trainer(
    model=peft_model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    args=transformers.TrainingArguments(
        output_dir=output_dir,
        warmup_steps=3,
        per_device_train_batch_size=6,
        gradient_accumulation_steps=2,
        max_steps=400,
        learning_rate=2.5e-5,
        optim="paged_adamw_8bit",
        logging_dir="./logs",        # Directory for storing logs
        logging_steps = 100,
        eval_strategy="steps",       # Evaluate the model every logging step
        eval_steps=50 ,              # Evaluate and save checkpoints every 50 steps
        do_eval=True,                # Perform evaluation at the end of training
        report_to=report_to
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

peft_model.config.use_cache = False  # Silence the warnings. Please re-enable for inference!
trainer.train()
peft_model.config.use_cache = True

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss,Validation Loss
50,No log,1.272774
100,1.206900,0.473889
150,1.206900,0.304236
200,0.320900,0.259088
250,0.320900,0.240316
300,0.244900,0.229892
350,0.244900,0.224866
400,0.219300,0.222856


Porównaj opowiedzi z bazowej, niedostrojonej wersji modelu (`model`) oraz dostrojonej wersji modelu (`peft_model`) dla kilku przykładowych elementów z testowego zbioru danych.

In [137]:
# # WRONG RESULTS -> ZOSTAWIONE W CELU WGLĄDU W BŁĘDY POPEŁNIONE PODCZAS PRÓBY ROZWIĄZANIA
# def compare_results():

#   number = random.randint(0, len(test_dataset) - 1)

#   print(f"{Fore.BLACK} BASE MODEL")
#   prompt = create_prompt_with_examples(test_dataset[number], 3, train_dataset)
#   generate_and_print_text(model, prompt, tokenizer, max_new_tokens=64)

#   print(f"{Fore.BLACK} PEFT_MODEL")
#   prompt = create_prompt_with_examples(test_dataset[number], 3, train_dataset)
#   generate_and_print_text(peft_model, prompt, tokenizer, max_new_tokens=128)


# for _ in range(4):
#   compare_results()


 BASE MODEL
EXAMPLE:
###
show me all the flights from philadelphia to cincinnati
###
{
    "fromloc.city_name": "philadelphia",
    "toloc.city_name": "cincinnati"
}
EXAMPLE:
###
show me the flights from boston to san francisco
###
{
    "fromloc.city_name": "boston",
    "toloc.city_name": "san francisco"
}
EXAMPLE:
###
could i have a listing of flights leaving boston going to baltimore leaving wednesday after 2100
###
{
    "depart_date.day_name": "wednesday",
    "depart_time.time": "2100",
    "fromloc.city_name": "boston",
    "toloc.city_name": "baltimore"
}
Convert the following text to a JSON format.
You can only use the JSON keys: ['fromloc.city_name', 'toloc.city_name', 'depart_date.day_name', 'depart_time.period_of_day', 'depart_date.day_number', 'depart_date.day_name', 'depart_date.month_name', 'depart_time.time', 'depart_date.relative', 'airline_name']. Not all keys are required.
- Format: Strictly use `{ "key": "value" }` with no extra text.
- Close JSON with `}`.
- If no

In [186]:
def compare_results(examples):

  number = random.randint(0, len(test_dataset) - 1)
  prompt = create_prompt_with_examples(test_dataset[number], examples, train_dataset)

  print(f"{Fore.BLACK} BASE MODEL")
  generate_and_print_text(model, prompt, tokenizer, max_new_tokens=80)

  print(f"{Fore.BLACK} PEFT_MODEL")
  generate_and_print_text(peft_model, prompt, tokenizer, max_new_tokens=80)


for _ in range(6):
  compare_results(0)

 BASE MODEL

Convert the following text to a JSON format.
You can only use the JSON keys: ['fromloc.city_name', 'toloc.city_name', 'depart_date.day_name', 'depart_time.period_of_day', 'depart_date.day_number', 'depart_date.day_name', 'depart_date.month_name', 'depart_time.time', 'depart_date.relative', 'airline_name']. Not all keys are required.
###
show flights from denver to houston
###

import json

def get_json(text):
    """
    Convert the following text to a JSON format.
    You can only use the JSON keys: ['fromloc.city_name', 'toloc.city_name', 'depart_date.day_name', 'depart_time.period_of_day', 'depart_
 PEFT_MODEL

Convert the following text to a JSON format.
You can only use the JSON keys: ['fromloc.city_name', 'toloc.city_name', 'depart_date.day_name', 'depart_time.period_of_day', 'depart_date.day_number', 'depart_date.day_name', 'depart_date.month_name', 'depart_time.time', 'depart_date.relative', 'airline_name']. Not all keys are required.
###
show flights from denver t

In [183]:
for _ in range(2):
  compare_results(1)

 BASE MODEL
EXAMPLE:
###
show me the flights from denver to san diego leaving after 5 pm
###
{
    "depart_time.time": "5 pm",
    "fromloc.city_name": "denver",
    "toloc.city_name": "san diego"
}
Convert the following text to a JSON format.
You can only use the JSON keys: ['fromloc.city_name', 'toloc.city_name', 'depart_date.day_name', 'depart_time.period_of_day', 'depart_date.day_number', 'depart_date.day_name', 'depart_date.month_name', 'depart_time.time', 'depart_date.relative', 'airline_name']. Not all keys are required.
###
i 'd like a flight from new york to los angeles on saturday morning on us air
###
{
    "depart_time.time": "5 am",
    "fromloc.city_name": "new york",
    "toloc.city_name": "los angeles",
    "depart_date.day_name": "saturday",
    "depart_date.day_number": "1",
    "
 PEFT_MODEL
EXAMPLE:
###
show me the flights from denver to san diego leaving after 5 pm
###
{
    "depart_time.time": "5 pm",
    "fromloc.city_name": "denver",
    "toloc.city_name": "san 

In [184]:
for _ in range(2):
  compare_results(3)

 BASE MODEL
EXAMPLE:
###
i 'd like to know the information from boston to philadelphia nonstop
###
{
    "fromloc.city_name": "boston",
    "toloc.city_name": "philadelphia"
}
EXAMPLE:
###
i would like to fly united airlines from washington dc to denver colorado
###
{
    "airline_name": "united airlines",
    "fromloc.city_name": "washington",
    "toloc.city_name": "denver"
}
EXAMPLE:
###
i would like a flight from baltimore to san francisco on twa with a stop in denver
###
{
    "fromloc.city_name": "baltimore",
    "toloc.city_name": "san francisco"
}
Convert the following text to a JSON format.
You can only use the JSON keys: ['fromloc.city_name', 'toloc.city_name', 'depart_date.day_name', 'depart_time.period_of_day', 'depart_date.day_number', 'depart_date.day_name', 'depart_date.month_name', 'depart_time.time', 'depart_date.relative', 'airline_name']. Not all keys are required.
###
show me the flights between los angeles and dallas
###
{
    "fromloc.city_name": "los angeles",
  

In [185]:
for _ in range(2):
  compare_results(5)

 BASE MODEL
EXAMPLE:
###
what flights are there arriving in chicago on continental after 11 pm
###
{
    "airline_name": "continental",
    "toloc.city_name": "chicago"
}
EXAMPLE:
###
i need a flight from philadelphia to dallas next wednesday in the morning
###
{
    "depart_date.day_name": "wednesday",
    "depart_time.period_of_day": "morning",
    "fromloc.city_name": "philadelphia",
    "toloc.city_name": "dallas"
}
EXAMPLE:
###
i need a return flight from philadelphia to boston
###
{
    "fromloc.city_name": "philadelphia",
    "toloc.city_name": "boston"
}
EXAMPLE:
###
when do planes leave boston for
###
{
    "fromloc.city_name": "boston"
}
EXAMPLE:
###
early morning flights between cincinnati and tampa
###
{
    "depart_time.period_of_day": "morning",
    "fromloc.city_name": "cincinnati",
    "toloc.city_name": "tampa"
}
Convert the following text to a JSON format.
You can only use the JSON keys: ['fromloc.city_name', 'toloc.city_name', 'depart_date.day_name', 'depart_time.per

#3.Wybrana metoda dostrojenia promptu

**Zadanie (dla chętnych)**

W tej części notatnika zaimplementuj trening z wykorzystaniem wybranej metody dostrajania promptu: prompt tuning, prefix tuning lub p-tuning z biblioteki PEFT Hugging Face.

Sprawdź jaki procent parametrów będzie podlegał optymalizacji, a jaki pozostanie zamrożony.

Wytrenuj model stosując zaimplementowaną metodę. Porównaj odpowiedzi z modelu bazowego i modelu dostrojonego na kilku przykładach ze zbioru testowego.

Wygeneruj ponownie zbiór treningowy i walidacyjny. Nie dodawaj polecenia "Convert the following text to a JSON format..." do promptów w zbiorze treningowym.
Zamiast dodawać polecenie w języku naturalnym wykorzystamy wirtualny prompt optymalizowany w procesie uczenia.

Ponownie wygeneruj stokenizowany zbiór treningowy i testowy korzystając z nowego promptu.

Utworzenie instancji modelu przygotowanej do dostrojenia metodą *prompt tuning*. W początkowym prompcie podlegającym optymalizacji warto umieścić listę kluczy JSON w wynikowym tekście (zmienna `attributes_to_keep`). Przykładowy początkowy prompt mógłby wyglądać następująco:

```
initial_prompt = f"{attributes_to_keep} xxx placeholder for virtual prompt xxx"
```


**Uwaga**: Instancję modelu dostosowaną do dostrajania utwórz w oparciu o kopię bazowego modelu model: `peft_model = get_peft_model(copy.deepcopy(model), config)`. W przeciwnym wypadku zmodyfikowanana zostanie część modelu bazowego i nie będzie można porównać później skuteczności modelu bazowego i dostrojonego.

In [ ]:
print(peft_model.print_trainable_parameters())

Dostrojenie modelu `peft_model` z wykorzystaniem klasy `Trainer`.

Odpowiedź z podstawowej wersji modelu `model`.

Odpowiedź z wersji z dostrojonym promptem modelu `peft_model`.

\\